In [11]:
# Common imports

import pandas as pd
import numpy as np

In [12]:
PATH_TO_FAR_ATLAS = 'data/far_atlas.csv'
PATH_TO_FVFB_DATA = 'data/fvfb_data.csv'

In [13]:
def load_data_locally():
    # as opposed to using Colab
    return pd.read_csv(PATH_TO_FAR_ATLAS), pd.read_csv(PATH_TO_FVFB_DATA)

In [14]:
df_fa, df_fvfb = load_data_locally()

In [15]:
def modify_fvfb(df):
    # Only get 2019 data
    df = df[df['Year'] == 2019]

    # Drop geopoint
    df.drop(columns=['Geopoint'])

    # Capitalize County
    capitalized = df['Locality'].apply(lambda x: x.title())
    df['Locality'] = capitalized

    return df

df_fvfb = modify_fvfb(df_fvfb)

C:\Users\thepe\AppData\Local\Temp\ipykernel_38660\1260576396.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Locality'] = capitalized


In [16]:
def modify_fa(df):
    # Only get VA data
    df = df[df['State'] == 'Virginia']

    # Drop Census Tract
    df.drop(columns=['CensusTract'])

    # Capitalize County
    capitalized = df['County'].apply(lambda x: x.title())
    df['County'] = capitalized

    return df

df_fa = modify_fa(df_fa)

C:\Users\thepe\AppData\Local\Temp\ipykernel_38660\1397294787.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['County'] = capitalized


In [17]:
fa_county_set = set(df_fa['County'].values)
fvfb_county_set = set(df_fvfb['Locality'].values)
county_intersection = fa_county_set.intersection(fvfb_county_set)
counties_only_in_fa = fa_county_set - county_intersection
counties_only_in_fvfb = fvfb_county_set - county_intersection

print('We have %d final counties, with only %d counties left out.' % (len(county_intersection), len(counties_only_in_fa) + len(counties_only_in_fvfb)))
print('Counties only in Far Atlas: %s' % counties_only_in_fa)
print('Counties only in VA dataset: %s' % counties_only_in_fvfb)

We have 130 final counties, with only 5 counties left out.
Counties only in Far Atlas: {'Manassas Park City', 'Falls Church City', 'Fairfax City'}
Counties only in VA dataset: {'Washington Dc', 'King And Queen'}


In [18]:
df_fa['PovertyRate'].value_counts()

0.0     22
1.8     18
2.3     18
7.9     17
3.2     17
        ..
31.9     1
43.9     1
22.2     1
27.0     1
22.1     1
Name: PovertyRate, Length: 367, dtype: int64

In [19]:
df_fvfb['Locality'].value_counts()

King And Queen County    36
Richmond City            31
Essex County             24
Hanover County           24
Pittsylvania County      24
                         ..
Winchester City          12
Gloucester County        12
Salem City               12
Washington Dc             3
Manassas City             2
Name: Locality, Length: 132, dtype: int64

In [20]:
df_fvfb['MTW Status'].value_counts()

Matching and updated    1830
Non-matching              27
Name: MTW Status, dtype: int64

In [36]:
def partition_fvfb_attribs(df_fvfb):
    unwanted_attribs = ['Year', 'Month','FIPS', 'Locality']
    fvfb_attribs_all = [attrib for attrib in df_fvfb.columns if attrib not in unwanted_attribs]
    fvfb_attribs_to_sum = ['Households Served','Individuals Served','Pounds of Food Distributed',
                           'Children Served via non-federal child nutrition programs',
                           'Pounds of food distributed via non-federal child nutrition progr',
                           'Virginia City/County Boundaries']
    fvfb_attribs_to_avg = ['LAT','LON']
    return fvfb_attribs_all, fvfb_attribs_to_sum, fvfb_attribs_to_avg

fvfb_attribs_all, fvfb_attribs_to_sum, fvfb_attribs_to_avg = partition_fvfb_attribs(df_fvfb)

def add_fvfb_mtw(df, df_fvfb):
    mtw_status_encoder = {'Non-matching':0, 'Matching and updated':1}
    mtw_status_decoder = ['Non-matching','Matching and updated']

    for county in df.index:
        rows = df_fvfb[df_fvfb['Locality'] == county]
        mtw = rows['MTW Status'].map(mtw_status_encoder)
        rounded_average = round(mtw.mean())
        df.loc[county, 'MTW Status'] = mtw_status_decoder[rounded_average]

    return df


def partition_fa_attribs(df_fa):
    # TODO: Figure out why POP2010 changed to Pop2010

    unwanted_attribs = ['State', 'CensusTract', 'County'] 
    fa_attribs_all = [attrib for attrib in df_fa.columns if attrib not in unwanted_attribs]
    fa_attribs_to_sum = ['Pop2010','OHU2010','NUMGQTRS','LAPOP1_10','LAPOP05_10','LAPOP1_20','LALOWI1_10','LALOWI05_10','LALOWI1_20']
    fa_attribs_flag = ['Urban','GroupQuartersFlag','LILATracts_1And10','LILATracts_halfAnd10','LILATracts_1And20','LILATracts_Vehicle',
                   'HUNVFlag','LowIncomeTracts','LA1and10','LAhalfand10','LA1and20','LATracts_half','LATracts1','LATracts10',
                   'LATracts20','LATractsVehicle_20']
    fa_attribs_to_avg = ['PCTGQTRS','PovertyRate','MedianFamilyIncome'] + fa_attribs_flag
    for attrib in fa_attribs_all:
        if attrib.endswith('share'):
            fa_attribs_to_avg.append(attrib)
            fa_attribs_to_sum.append(attrib[:-5])
        if attrib.startswith('Tract'):
            fa_attribs_to_sum.append(attrib)
            
    missed_attribs = set(fa_attribs_all) - set(fa_attribs_to_sum).union(set(fa_attribs_to_avg))
            
    if missed_attribs:
        raise Exception('We missed these attribs in fa: %s' % missed_attribs)

    return fa_attribs_all, fa_attribs_to_sum, fa_attribs_to_avg, fa_attribs_flag

fa_attribs_all, fa_attribs_to_sum, fa_attribs_to_avg, fa_attribs_flag = partition_fa_attribs(df_fa)


def add_df_fvfb(df, df_fvfb):
    for county in df.index:
        rows = df_fvfb[df_fvfb['Locality'] == county]
        for attrib in fvfb_attribs_to_sum:
            summed = rows[attrib].sum()
            df.loc[county, attrib] = summed
        for attrib in fvfb_attribs_to_avg:
            average = rows[attrib].mean()
            df.loc[county, attrib] = average

    df = add_fvfb_mtw(df, df_fvfb)

    return df


def add_df_fa(df, df_fa):
    for county in df.index:
        
        rows = df_fa[df_fa['County'] == county]
        for attrib in fa_attribs_to_sum:
            summed = rows[attrib].sum()
            df.loc[county, attrib] = summed
        for attrib in fa_attribs_to_avg:
            result = rows[attrib].mean()
            if attrib in fa_attribs_flag:
                result = round(result)
            df.loc[county, attrib] = result

    return df


def merge_tables(df_fvfb, df_fa):
    df = pd.DataFrame(index=sorted(county_intersection))

    df = add_df_fvfb(df, df_fvfb)
    df = add_df_fa(df, df_fa)
     
    return df

df = merge_tables(df_fvfb, df_fa)


C:\Users\thepe\AppData\Local\Temp\ipykernel_38660\1164389558.py:79: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df.loc[county, attrib] = result
C:\Users\thepe\AppData\Local\Temp\ipykernel_38660\1164389558.py:79: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df.loc[county, attrib] = result
C:\Users\thepe\AppData\Local\Temp\ipykernel_38660\1164389558.py:79: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all column

Accomack County     1920.0
Albemarle County    1844.0
Alexandria City     3083.0
Alleghany County     909.0
Amelia County        509.0
Name: TractSNAP, dtype: float64

In [37]:
df.describe()

,Households Served,Individuals Served,Pounds of Food Distributed,Children Served via non-federal child nutrition programs,Pounds of food distributed via non-federal child nutrition progr,Virginia City/County Boundaries,LAT,LON,Pop2010,OHU2010,...,laseniors20share,lawhite20share,lablack20share,laasian20share,lanhopi20share,laaian20share,laomultir20share,lahisp20share,lahunv20share,lasnap20share
count,130.000000,130.000000,1.300000e+02,130.000000,130.000000,130.000000,130.000000,130.000000,1.300000e+02,130.000000,...,1.00,1.00,1.00,1.00,1.0,1.0,1.00,1.0,5.000000,5.000000
mean,12377.046154,43750.869231,9.190581e+05,1734.369231,18872.530769,932.953846,37.535942,-78.540582,6.116811e+04,23370.023077,...,6.22,25.11,0.09,0.04,0.0,0.0,0.06,0.0,0.400000,0.322000
std,15880.033959,84878.083314,1.584730e+06,4152.647552,39871.172813,609.923275,0.687624,1.737099,1.181063e+05,43181.704666,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.894427,0.720014
min,0.000000,0.000000,0.000000e+00,0.000000,0.000000,12.000000,36.583334,-83.130112,2.321000e+03,1081.000000,...,6.22,25.11,0.09,0.04,0.0,0.0,0.06,0.0,0.000000,0.000000
25%,3524.000000,10661.250000,2.219574e+05,0.000000,0.000000,423.000000,36.979008,-79.691663,1.429425e+04,5770.750000,...,6.22,25.11,0.09,0.04,0.0,0.0,0.06,0.0,0.000000,0.000000
50%,6966.000000,21596.500000,3.999801e+05,0.000000,0.000000,917.500000,37.365890,-78.224722,2.563900e+04,10102.000000,...,6.22,25.11,0.09,0.04,0.0,0.0,0.06,0.0,0.000000,0.000000
75%,13747.750000,41640.000000,9.410836e+05,1224.000000,20557.750000,1377.000000,38.011058,-77.230678,5.486750e+04,22827.250000,...,6.22,25.11,0.09,0.04,0.0,0.0,0.06,0.0,0.000000,0.000000
max,110642.000000,784495.000000,1.401186e+07,27759.000000,245377.000000,2856.000000,39.203637,-75.757807,1.081726e+06,391627.000000,...,6.22,25.11,0.09,0.04,0.0,0.0,0.06,0.0,2.000000,1.610000


In [40]:
def write_df_to_file(df, file_path):
    df.to_csv(file_path, index_label='County', index=True)

write_df_to_file(df, 'data/merged_data.csv')